In [2]:
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)

In [3]:
import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=8)

In [5]:
# reate logger - nocer formatting
import logging

logger = logging.getLogger()
logging._warn_preinit_stderr = 0
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s: %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.handlers = [ch]
logger.info("Logger started.")

2020-12-03 20:14:33,219: Logger started.


In [6]:
import sqlite3 
import numpy as np

def adapt_array(arr):
    """
    Save Numpy array to SqLite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    """ 
    Load Numpy array from Sqlite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)
data_path = os.path.join(os.getcwd(), "samplesGeneration", "prepared_data.bd")
db = sqlite3.connect(data_path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)

In [7]:
import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 

True

In [8]:
import io
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

db_cursor = db.cursor()
sql_query = "SELECT COALESCE(MAX(id)+1, 0) FROM data"
db_cursor.execute(sql_query)
number_of_samples = db_cursor.fetchone()[0]
print("number of samples: ", number_of_samples)

sql_query = "SELECT x, y FROM data WHERE id == 1"
db_cursor.execute(sql_query)
print(db_cursor.fetchone())

number of samples:  1703921
(array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  2.00099993e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  9.08880856e-01, -4.17055860e-01,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  4.00199986e+00,
        0.00000000e+00,  0.00000000e+00,  1.00000000e+00, -7.58108174e-01,
       -6.52128819e-01,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        6.00299978e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
       -2.76533943e-01,  9.61004151e-01,  0.00000000e+00,  1.00000000e+00,
        1.66296152e+00,  0.00000000e+00,  1.00000000e+00,  7.48810198e-05,
        2.11998690e-06, -8.31112775e-05,  7.48810198e-05,  9.99999997e-01,
        2.11998690e-06,  1.00000000e+00, -8.31112774e-05,  9.99999997e-01,
       -5.57474632e-05,  1.99994695e+00,  2.09227568e-04, -5.57474632e-

In [11]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras from SQLite database'
    def __init__(self, path, num_features, end_index=None, start_index=1, batch_size=25,
                 shuffle=True, dim=(117, 1), dim_y=(114, 1), column="voxels"):
        'Initialization'
        sqlite3.register_adapter(np.ndarray, adapt_array)
        sqlite3.register_converter("array", convert_array)
        self.path = path
        self.dim = dim
        self.dim_y = dim_y
        self.db = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
        self.db_cursor = self.db.cursor()
        self.N = num_features
        self.start_index = start_index
        if end_index is None:
            end_index =  num_features + start_index
        self.end_index = end_index
        self.column = column
        self.batch_size = int(batch_size)
        self.shuffle = shuffle
        self.sample_index = np.arange(self.start_index, self.end_index)
        self.on_epoch_end()
        self.db.close()


    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.N / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data - generates indexes of the batch'
        # Generate indexes of the batch
        samples_batch = np.arange((index) * self.batch_size, (index+1) * self.batch_size)

        # Generate data
        while True:
            try:
                x, y = self.__data_generation(samples_batch)
                break
            except ValueError:
                continue

        return  x, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            np.random.shuffle(self.sample_index)

    def __data_generation(self, samples_batch):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        x = np.empty((self.batch_size, *self.dim))
        y = np.empty(shape=(self.batch_size, *self.dim_y))
        inds = self.sample_index[samples_batch]
        sqlite3.register_adapter(np.ndarray, self.adapt_array)
        sqlite3.register_converter("array", self.convert_array)
        db = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES)
        db_cursor = db.cursor()
        sql_query = "SELECT x, y FROM data WHERE id in ({index})".\
            format(column=self.column, index=','.join(str(ind) for ind in inds))
        db_cursor.execute(sql_query)
        for i in range(self.batch_size):
            line = db_cursor.fetchone()
            x[i, :] = line[0].reshape(self.dim)
            y[i] = line[1].reshape(self.dim_y)
        # x.reshape(self.batch_size, *self.dim, 1)
        y = y.reshape(self.batch_size, *self.dim_y)
        return x, y
    
    def adapt_array(self, arr):
        """
        Save Numpy array to SqLite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO()
        np.save(out, arr)
        out.seek(0)
        return sqlite3.Binary(out.read())

    def convert_array(self, text):
        """ 
        Load Numpy array from Sqlite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO(text)
        out.seek(0)
        return np.load(out)
    
test_train_ratio = 0.1

test_size = int(test_train_ratio * number_of_samples)
training_generator = DataGenerator(data_path, num_features=number_of_samples - test_size, batch_size=64, dim=(117, 1), dim_y=(36, 1))
validation_generator = DataGenerator(data_path, num_features=test_size, start_index=number_of_samples - test_size, batch_size=64, dim=(114, 1), dim_y=(36, 1))

In [10]:
# Try learning for all of verts:

verts = range(14)
pivot_vert = 3

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import LSTM
from collections import deque
from keras.callbacks import EarlyStopping
import time


def create_and_train_network(generator, val_generator, epohs=100):
    def create_model():
        model = Sequential()
        #model.add(LSTM(100, input_shape=generator.dim, return_sequences=True))
        #model.add(Dropout(0.2))
        model.add(LSTM(100, input_shape=generator.dim, return_sequences=False))
        model.add(Dropout(0.2))
        #model.add(Flatten())
        model.add(Dense(np.prod(generator.dim_y), activation='linear'))
        model.add(Reshape((np.prod(generator.dim_y), 1)))
        model.compile(loss='mae', optimizer='adam')
        return model

    logger.info("Create model...")
    model = create_model()
    model.summary()
    logger.info("Fit model..")
    
    network = model.fit_generator(training_generator, verbose=1, validation_data=validation_generator, 
                                              epochs=epohs, callbacks=[early_stopping])
    logger.info("Summary")
    model.summary()
    model.save(os.path.join(models_dir, "model_all_points_{}".format(sample_length)))


logger.info("start")
create_and_train_network(training_generator, validation_generator)

2020-12-03 20:14:57,454: start
2020-12-03 20:14:57,456: Create model...
2020-12-03 20:14:58,161: Fit model..


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                3636      
_________________________________________________________________
reshape_1 (Reshape)          (None, 36, 1)             0         
Total params: 44,436
Trainable params: 44,436
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
23960/23961 [============================>.] - ETA: 0s - loss: 0.1404

ValueError: cannot reshape array of size 117 into shape (114,1)